In [1]:
!pip install langgraph


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip show langgraph

In [ ]:

import os

# Option A: already set in your shell
# os.environ['OPENAI_API_KEY']

# Option B: set here for this notebook session
os.environ['OPENAI_API_KEY'] = "add api key here"

print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))


OPENAI_API_KEY set: True


In [3]:
# ensure your repo is importable if not installed
import os, sys, json


# ✅ OpenAI key (uncomment to set in-notebook)
# os.environ["OPENAI_API_KEY"] = "sk-..."

from src.db_crawl_agents.workflow.feature_identification import OrchestratorGraph
from src.db_crawl_agents.contracts.feature_orchestrator.feature_orchestrator import UserQuery, Feedback

# your runnable + adapter
from src.db_crawl_agents.utils.runnable_chat_model import RunnableChatModel
from src.db_crawl_agents.utils.feature_orchestrator.LLMAdapter import RunnableLLMAdapter

# provider-native chat model (your wrapper)
from src.db_crawl_agents.llms.openai_integration.chat_openai import OpenAIChat

def jprint(x): print(json.dumps(x, indent=2, default=str))

In [ ]:
# pick a good JSON-capable model

base = OpenAIChat(model="gpt-4o")

# wrap in your Runnable and the orchestrator-facing adapter
runnable = RunnableChatModel(base)
llm = RunnableLLMAdapter(runnable, default_max_features=5)

# build the orchestrator graph
og = OrchestratorGraph(llm=llm, max_features=5)
print("Graph ready.")

Graph ready.


In [5]:
query = UserQuery(text="I need features to measure purchasing intensity over the last 30 days.")
draft_resp = og.run(query=query, feedback=None, finalize=False)  # stop at draft
jprint(draft_resp)

propose features raww text, {'normalized_user_intent': 'Analyze and quantify purchasing activity over the past 30 days.', 'proposed_features': [{'id': 'feat.TOTAL_PURCHASES', 'name': 'TOTAL_PURCHASES', 'business_title': 'Total Purchases', 'description': 'The total number of purchases made in the last 30 days.', 'target_grain': 'customer', 'temporal_scope': 'last 30 days', 'value_type': 'integer', 'valid_values': None, 'acceptance_criteria': 'Must match the sum of all purchase records for the period.', 'dependencies': ['purchase records'], 'complexity_hint': {'difficulty': 'low', 'drivers': ['aggregation of purchase records']}, 'example_row': {'customer_id': '12345', 'total_purchases': 15}, 'notes': ['Ensure data includes all purchase channels.'], 'linked_with': [], 'source_systems': ['sales_database']}, {'id': 'feat.AVERAGE_PURCHASE_VALUE', 'name': 'AVERAGE_PURCHASE_VALUE', 'business_title': 'Average Purchase Value', 'description': 'The average value of purchases made in the last 30 da

In [7]:
fb = Feedback(
    text="Rename AOV_30D to MEAN_ORDER_VALUE_30D and exclude refunded orders.",
    update={"AOV_30D": {"name": "MEAN_ORDER_VALUE_30D", "notes": ["Exclude refunded orders."]}},
)

refined_resp = og.run(query=query, feedback=fb, finalize=False)
jprint(refined_resp)



propose features raww text, {'normalized_user_intent': 'Measure the purchasing intensity of customers over the last 30 days.', 'proposed_features': [{'id': 'feat.TOTAL_PURCHASES', 'name': 'TOTAL_PURCHASES', 'business_title': 'Total Purchases', 'description': 'Total number of purchases made by a customer in the last 30 days.', 'target_grain': 'customer_id', 'temporal_scope': 'last 30 days', 'value_type': 'integer', 'valid_values': None, 'acceptance_criteria': 'Sum of all purchase transactions for a customer in the specified period.', 'dependencies': ['purchase_transactions'], 'complexity_hint': {'difficulty': 'low', 'drivers': ['transaction data availability']}, 'example_row': {'customer_id': 123, 'total_purchases': 5}, 'notes': ['Ensure all purchase transactions are captured.'], 'linked_with': [], 'source_systems': ['transaction_system']}, {'id': 'feat.AVERAGE_PURCHASE_VALUE', 'name': 'AVERAGE_PURCHASE_VALUE', 'business_title': 'Average Purchase Value', 'description': 'Average value of

In [8]:
final_resp = og.run(query=query, feedback=fb, finalize=True)
jprint(final_resp)

propose features raww text, {'normalized_user_intent': 'Measure the purchasing intensity of customers over the last 30 days.', 'proposed_features': [{'id': 'feat.TOTAL_PURCHASE_AMOUNT', 'name': 'TOTAL_PURCHASE_AMOUNT', 'business_title': 'Total Purchase Amount', 'description': 'The total monetary value of purchases made by a customer over the last 30 days.', 'target_grain': 'customer_id', 'temporal_scope': 'last 30 days', 'value_type': 'decimal', 'valid_values': None, 'acceptance_criteria': 'Sum of all purchase transactions for each customer in the specified period.', 'dependencies': ['purchase_transactions'], 'complexity_hint': {'difficulty': 'low', 'drivers': ['aggregation of transaction data']}, 'example_row': {'customer_id': '12345', 'total_purchase_amount': 250.75}, 'notes': ['Ensure all transactions are captured within the specified period.'], 'linked_with': [], 'source_systems': ['transaction_database']}, {'id': 'feat.NUMBER_OF_PURCHASES', 'name': 'NUMBER_OF_PURCHASES', 'business